In [ ]:
import pandas as pd
from S1000_model import S1000_comp_model
from names import DICT_COMP_SIGNALKEY
import numpy as np
import matplotlib.pyplot as plt
from time_templates.datareader.get_data import fetch_MC_data_from_tree
from time_templates.templates.universality.S1000_model import set_Rmu_df
from time_templates.utilities.plot import plot_profile_1d, plot_hist, add_identity
from time_templates.utilities.misc import histedges_equalN
from time_templates.utilities.fitting import plot_fit_curve

from time_templates.templates.universality.names import DICT_COMP_COLORS, eMUON, eEM_HAD, eEM_PURE, eEM_MU, DICT_COMP_LABELS, DICT_COMP_MARKERS, DICT_COMP_SIGNALKEY
from S1000_model import S1000_comp_model, sec_theta_func, SEC_THETA_CORR_COMP, Rmu_func
from matplotlib.lines import Line2D
SAVE = True
BOOTSTRAPS = 0
plt.style.use('thesis')

if SAVE:
    BOOTSTRAPS = 0
    
gcm2 = "\\mathrm{g/cm^2}"
label_DX = f"$\\Delta X \, [{gcm2}]$"
label_VEM = "$\\rm Signal\, [\\rm VEM]$"

DICT_FRAC_LABELS = {
    comp: f"$S_{{{label}}}(1000) / \\widehat{{S}}_{{{label}}}(1000)$"
    for comp, label in DICT_COMP_LABELS.items()
}
DICT_FRAC_MEAN_LABELS = {
    comp: f"$\\langle S_{{{label}}}(1000) / \\widehat{{S}}_{{{label}}}(1000) \\rangle$"
    for comp, label in DICT_COMP_LABELS.items()
}

df = pd.read_pickle("/home/mart/auger/data/time_templates/df_shower_size.pl")
thesis_fig_path = "/home/mart/auger/thesis/thesis_overleaf/figs/"
# %matplotlib notebook

In [ ]:
df_p = df.query("primary == 'proton'")
df_gb = df_p.groupby(['MClgE_bin', 'MCDX_bin'])
df_mean = df_gb.mean()
df_std = df_gb.std()
df_n = df_gb.count()

df_mean = df_mean.loc[19.5]
df_err = df_std.loc[19.5]/np.sqrt(df_n.loc[19.5])
x = df_mean['MCDX_1000']
f, ax = plt.subplots(1, figsize=(4, 3.3))
xspace = np.linspace(x.min(), x.max(), 200)
for comp, signal_key in DICT_COMP_SIGNALKEY.items():
    y = df_mean[signal_key].values

    yerr = df_err[signal_key].values
    mask = np.isfinite(x * y * yerr) & (yerr > 0)

    fitfunc = lambda DX, Sref, DXmax, lam: S1000_func(
        DX, Sref, DXmax, DX0_COMP[comp], lam
    )
    p0 = [10, 300, 200]

    comp_label = DICT_COMP_LABELS[comp]

    ax.errorbar(x, y, yerr=yerr, marker=DICT_COMP_MARKERS[comp], color=DICT_COMP_COLORS[comp], ls='')
    ax.plot(xspace, S1000_comp_model(19.45, xspace, np.deg2rad(38), comp=comp), color=DICT_COMP_COLORS[comp])
    
ax.set_ylabel('Signal [VEM]')
ax.set_xlabel('$\Delta X(r=1000\, \\rm m, \psi=90^\\circ)\, [\\rm g/cm^2]$')
ax.set_yscale('log')
labels = [f"${lab}$" for lab in DICT_COMP_LABELS.values()]

handles = [
    Line2D([0], [0], color=DICT_COMP_COLORS[key], marker=DICT_COMP_MARKERS[key])
    for key in DICT_COMP_LABELS.keys()
]
ax.legend(handles, labels, loc=3)

if SAVE:
    plt.savefig(thesis_fig_path+"/DXmax_vs_signal_example.pdf")

In [ ]:

Rmu_mean_Fe = df.query('primary == "iron"')["Rmu"].mean()
print("Iron Rmu mean", Rmu_mean_Fe)
df['total_pred'] = 0
df["Rmu"] = 1
df["Rmu"].loc[df["primary"] == "proton"] = 1
df["Rmu"].loc[df["primary"] == "iron"] = Rmu_mean_Fe  # ~1.33
for comp, signal_key in DICT_COMP_SIGNALKEY.items():
    df[comp + "_pred_no_sct_corr"] = S1000_comp_model(df['MClgE'].values, 
                                                     df['MCDX_1000'].values,
                                                     np.arccos(1/1.3),
                                                     1,
                                                     comp,
                                                     sec_theta_corr_comp=None)
    df[comp + "_ratio_no_sct_corr"] = df[signal_key]/df[comp + "_pred_no_sct_corr"]
    df[comp + "_pred_no_Rmu_corr"] = S1000_comp_model(df['MClgE'].values, df['MCDX_1000'].values,
                                                     df['MCTheta'].values,
                                                     1,
                                                     comp)
    df[comp + "_ratio_no_Rmu_corr"] = df[signal_key]/df[comp + "_pred_no_Rmu_corr"]
    df[comp + "_pred"] = S1000_comp_model(df['MClgE'].values, df['MCDX_1000'].values,
                                                     df['MCTheta'].values,
                                                     df['Rmu'].values,
                                                     comp)
    df[comp + "_ratio"] = df[signal_key]/df[comp + "_pred"]
    df['total_pred'] += df[comp+"_pred"]
    
df['total_ratio'] = df['WCDTotalSignal']/df['total_pred']

df_gb = df.groupby("EventId")
df_ = df_gb.mean()
df_["primary"] = df_gb["primary"].min()

df_p = df_.query("primary == 'proton'")
df_Fe = df_.query("primary == 'iron'")

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(6, 6))
axes = axes.flatten()

bins = histedges_equalN(df['MCSecTheta'], 15)
x = np.linspace(1, 2, 100)
for (comp, signal_key), ax in zip(DICT_COMP_SIGNALKEY.items(), axes):
    ax, (x, y, yerr) = plot_profile_1d(
        df_p["MCSecTheta"],
        df_p[comp + "_ratio_no_sct_corr"],
        ax=ax,
        bins=bins,
        color=DICT_COMP_COLORS[comp],
        stat="mean",
        bootstraps=BOOTSTRAPS,
    )
#     ax.plot(x, sec_theta_func(x, *SEC_THETA_CORR_COMP[comp]), color=DICT_COMP_COLORS[comp])
    ax, (fitp, _, _, _) = plot_fit_curve(
        x,
        y,
        yerr=yerr,
        func=sec_theta_func,
        ax=ax,
        ebar_kws=dict(alpha=0, color=DICT_COMP_COLORS[comp]),
        smoother_x=True,
    )
    ax.set_xlabel("$\\sec{\\theta}$")
    ax.set_ylabel(DICT_FRAC_MEAN_LABELS[comp])
    ax.legend(fontsize=8)
    ax.set_ylim([0.8, 1.2])
    ax.axhline(1, ls="--", color="k")
    ax.axhspan(0.95, 1.05, color="k", alpha=0.1)
    ax.set_xlim([1, 2])
    ax_top = ax.twiny()
    ax_top.set_xlim(ax.get_xlim())
    new_tick_locations = np.array([1, 1.15, 1.3, 1.55, 2])
    ax_top.set_xticks(new_tick_locations)
    new_tick_labels = np.rad2deg(np.arccos(1 / new_tick_locations))
    ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
    #     ax_top.set_xlabel('$\\theta$ [deg]')
    ax_top.tick_params(
        axis="x",
        which="minor",
        direction="in",
        top=False,
        labeltop=True,
        bottom=False,
        labelbottom=False,
    )
plt.subplots_adjust(hspace=0.33, wspace=0.3)
# plt.tight_layout()
if SAVE:
    plt.savefig(thesis_fig_path + "/signal_ratio_sec_theta_dependence.pdf")


In [ ]:

# For fitting
bins = np.linspace(0.6, 1.5, 11)

f, axes = plt.subplots(1, 3, figsize=(6., 2.5))
axes = axes.flatten()

iax = 0

for comp in DICT_COMP_SIGNALKEY.keys():
    ax = axes[iax]
    if comp != eMUON:
        iax += 1

    ax.scatter(df_p[eMUON+"_ratio_no_Rmu_corr"], df_p[comp + "_ratio_no_Rmu_corr"], color="b", alpha=0.01, marker=".")
    ax.scatter(df_Fe[eMUON+"_ratio_no_Rmu_corr"], df_Fe[comp + "_ratio_no_Rmu_corr"], color="r", alpha=0.01, marker=".")
    ax, (x, y, yerr) = plot_profile_1d(
        df_[eMUON + "_ratio_no_Rmu_corr"],
        df_[comp + "_ratio_no_Rmu_corr"],
        ax=ax,
        bins=bins,
        marker="",
        alpha=0,
        stat="mean",
        bootstraps=BOOTSTRAPS,
    )
    mask = np.isfinite(x * y * yerr) & (yerr > 0) & (yerr/y < 0.1)
    ax, (pfit, pfit_err, chi2, ndf) = plot_fit_curve(
        x[mask],
        y[mask],
        yerr=yerr[mask],
        func=Rmu_func,
        ax=ax,
        ebar_kws=dict(color="k", ls="", marker="o", alpha=1),
    )

#     ax, (x, y, yerr) = plot_profile_1d(
#         df_p["Rmu"],
#         df_p[comp + "_ratio_no_Rmu_corr"],
#         ax=ax,
#         bins=bins, #histedges_equalN(df_p["Rmu"], 6),
#         color="b",
#         mec='k',
#     )
#     ax, (x, y, yerr) = plot_profile_1d(
#         df_Fe["Rmu"],
#         df_Fe[comp + "_ratio_no_Rmu_corr"],
#         ax=ax,
#         bins=bins, #histedges_equalN(df_Fe["Rmu"], 6),
#         color="r",
#         mec='k',
#     )

    if comp == eEM_PURE:
        ax.axhline(1, ls=":", color="k")
    else:
        add_identity(ax, color="k", ls=":")
    ax.set_xlabel("$R^\\mu$")
    ax.set_ylabel(DICT_FRAC_LABELS[comp])
#     ax.annotate(
#         xy=(0.1, 0.95),
#         text=f"$\\alpha_{{{DICT_COMP_LABELS[comp]}}} = {pfit[0]:.3f} \\pm {pfit_err[0]*np.sqrt(chi2/ndf):.3f}$",
#         fontsize=8,
#         xycoords="axes fraction",
#         ha="left",
#         va="top",
#     )
    #     ax.legend()
    ax.set_ylim([0.55, 1.8])
    ax.set_xlim([0.55, 1.55])
    if comp == eMUON:
        ax.clear()

plt.subplots_adjust(wspace=0.5)
if SAVE:
    plt.savefig(thesis_fig_path + "signal_ratio_Rmu_dependence.pdf")


In [ ]:
bins = np.linspace(0.4, 1.6, 40)
df_p_ = df_p.query("MCCosTheta > 0.6 & MClgE > 19.4 & MClgE < 19.6")
df_Fe_ = df_Fe.query("MCCosTheta > 0.6 & MClgE > 19.4 & MClgE < 19.6")

f, ax = plt.subplots(1, figsize=(2.5, 2.5))
plot_hist(
    df_p_["total_ratio"], color="b", bins=bins, histtype="step", fit_norm=False, ax=ax, density=True, lw=0.6,
)
plot_hist(
    df_Fe_["total_ratio"], color="r", bins=bins, histtype="step", fit_norm=False, ax=ax, density=True, lw=0.6
)
ax.axvline(1, ls="--", color="k")

ax.legend(ncol=1, fontsize=6)
ax.axvline(1, ls="--", color="k")
ax.set_xlabel(f"$S_{{\\rm total}}(1000) / \\widehat{{S}}_{{\\rm total}}(1000)$")
ax.set_ylabel("probability density")
# ax.set_yscale('log')
# ax.set_ylim([1e-1, ])
if SAVE:
    plt.savefig(thesis_fig_path+"total_signal_ratio_hist.pdf")


In [ ]:
f, ax = plt.subplots(1, figsize=(2.5, 2.5))

bins = histedges_equalN(df['MCSecTheta'], 15)
plot_profile_1d(
    df_p["MCSecTheta"],
    df_p["total_ratio"],
    bins=bins,
    ax=ax,
    color="b",
    bootstraps=BOOTSTRAPS,
    markersize=3,
)
plot_profile_1d(
    df_Fe["MCSecTheta"],
    df_Fe["total_ratio"],
    bins=bins,
    ax=ax,
    color="r",
    bootstraps=BOOTSTRAPS,
    markersize=3
)
ax.set_ylim([0.79, 1.2])
ax.axhspan(0.95, 1.05, color="k", alpha=0.1)
ax.axhline(1, ls="--", color="k")
ax.set_ylabel("$\\langle S_{{\\rm total}}(1000) / \\widehat{{S}}_{{\\rm total}}(1000) \\rangle$")
ax.set_xlabel("$\\sec{\\theta}$")

ax_top = ax.twiny()
ax_top.set_xlim(ax.get_xlim())
new_tick_locations = np.array([1, 1.15, 1.3, 1.55, 2])
ax_top.set_xticks(new_tick_locations)
new_tick_labels = np.rad2deg(np.arccos(1 / new_tick_locations))
ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
ax_top.tick_params(
    axis="x",
    which="minor",
    direction="in",
    top=False,
    labeltop=True,
    bottom=False,
    labelbottom=False,
)
ax.axvspan(1/0.6, 2, color="k", hatch="//", fc='none')
ax.set_xlim([1, 2])
if SAVE:
    plt.savefig(thesis_fig_path+"total_signal_ratio_profile_all_corrected_sec_theta.pdf")


In [ ]:
f, ax = plt.subplots(1, figsize=(2.5, 2.5))
df_p_ = df_p.query('MCCosTheta > 0.6')
df_Fe_ = df_Fe.query('MCCosTheta > 0.6')
bins = np.arange(18.5, 20.1, 0.1) #histedges_equalN(df['MCSecTheta'], 15)
plot_profile_1d(
    df_p_["MClgE"],
    df_p_["total_ratio"],
    bins=bins,
    ax=ax,
    color="b",
    bootstraps=BOOTSTRAPS,
    markersize=3,
)
plot_profile_1d(
    df_Fe_["MClgE"],
    df_Fe_["total_ratio"],
    bins=bins,
    ax=ax,
    color="r",
    bootstraps=BOOTSTRAPS,
    markersize=3
)

ax_top.set_xscale('log')

ax.set_ylim([0.79, 1.2])
ax.axhspan(0.95, 1.05, color="k", alpha=0.1)
ax.axhline(1, ls="--", color="k")
ax.set_ylabel("$\\langle S_{{\\rm total}}(1000) / \\widehat{{S}}_{{\\rm total}}(1000) \\rangle$")
ax.set_xlabel('$\\lg{E/\\rm eV}$')
if SAVE:
    plt.savefig(thesis_fig_path+"total_signal_ratio_profile_all_corrected_lgE.pdf")


In [ ]:
f, ax = plt.subplots(1)
df_p_ = df_p.groupby('EventId').mean()
df_Fe_ = df_Fe.groupby('EventId').mean()
bins=np.linspace(0.35, 1.9, 50)
plot_hist(df_p_['muon_ratio_no_Rmu_corr'], ax=ax, bins=bins, histtype='step', color='b', density=True)
plot_hist(df_Fe_['muon_ratio_no_Rmu_corr'], ax=ax, bins=bins, histtype='step', color='r', density=True)
ax.legend(loc=2)
ax.set_xlabel('$R^\\mu$')
ax.set_ylabel('probability density')

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(6, 6))
axes = axes.flatten()

bins = histedges_equalN(df["MCSecTheta"], 15)
for (comp, signal_key), ax in zip(DICT_COMP_SIGNALKEY.items(), axes):
    ax, (x, y, yerr) = plot_profile_1d(
        df_p["MCSecTheta"],
        df_p[comp + "_ratio"],
        ax=ax,
        bins=bins,
        color="b",
        bootstraps=BOOTSTRAPS,
    )
    ax, (x, y, yerr) = plot_profile_1d(
        df_Fe["MCSecTheta"],
        df_Fe[comp + "_ratio"],
        ax=ax,
        bins=bins,
        color="r",
        bootstraps=BOOTSTRAPS,
    )

    ax.set_ylim([0.8, 1.2])
    ax.axhspan(0.95, 1.05, color="k", alpha=0.1)
    ax.axhline(1, ls="--", color="k")
    ax.set_ylabel(DICT_FRAC_MEAN_LABELS[comp])
    ax.set_xlabel("$\\sec{\\theta}$")

    ax_top = ax.twiny()
    ax_top.set_xlim(ax.get_xlim())
    new_tick_locations = np.array([1, 1.15, 1.3, 1.55, 2])
    ax_top.set_xticks(new_tick_locations)
    new_tick_labels = np.rad2deg(np.arccos(1 / new_tick_locations))
    ax_top.set_xticklabels([f"${int(round(l, 0))}^\\circ$" for l in new_tick_labels])
    ax_top.tick_params(
        axis="x",
        which="minor",
        direction="in",
        top=False,
        labeltop=True,
        bottom=False,
        labelbottom=False,
    )
plt.subplots_adjust(hspace=0.33, wspace=0.3)
if SAVE:
    plt.savefig(
        thesis_fig_path+"signal_ratio_comp_profile_all_corrected_sec_theta.pdf")


In [ ]:
np.sqrt(0.185)

In [ ]:
np.sqrt()